In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [18]:
BASE_PATH = "/home/hrishi/SOP/Gravitational Wave Detection Using Deep Learning/models/debugging/Missing Data Imputation/Test 1/data/"
MASK_FILE = BASE_PATH + "mask.csv"
DATA_FILE = BASE_PATH + "data.csv"
WEIGHTS_PATH = BASE_PATH + "weights.pth"
NOISE_DURATION = 600 # in sec
SAMPLING_FREQUENCY = 4096
TOTAL_SAMPLES = 1000
NO_OF_TRAINING_SAMPLES = 1000
NO_OF_TESTING_SAMPLES = 1000

# Hyperparameters
INPUT_SIZE = 1
HIDDEN_SIZE = 50
DEVICE = "cuda"
LR = 1e-3
LOAD_WEIGHTS = False
EPOCHS = 10

In [5]:
class imputationLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstmcell = nn.LSTMCell(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, input_size)
        self.mse_loss = nn.MSELoss()

    def forward(self, x, mask):
        hx = torch.randn(x.size()[1], self.hidden_size)
        cx = torch.randn(x.size()[1], self.hidden_size)
        next_input = torch.randn(x.size()[1], self.input_size)
        output = [[] for i in range(x.size()[1])]
        loss = 0

        for i in range(-1, x.size()[0]):
            if i == -1:
                hx, cx = self.lstmcell(next_input, (hx, cx))

            else:
                hx, cx = self.lstmcell(next_input, (hx, cx))

            if i != x.size()[0]-1:
                tmp_mask = torch.tensor(mask[:, i+1], dtype=bool)
                inv_tmp_mask = torch.tensor(~mask[:, i+1], dtype=bool)
                tmp_out = self.linear(hx)

                next_input = torch.mul(tmp_out, tmp_mask) + torch.mul(x[i+1], inv_tmp_mask) 
                loss += self.mse_loss(next_input, x[i+1])

                with torch.no_grad():
                    for j in range(len(mask[:, i+1])):
                        if mask[j, i+1]:
                            output[j].append(tmp_out[j, 0])
        
        output = torch.stack(output, dim=0)
        return output, loss

In [ ]:
start_time = pd.read_csv(MASK_FILE, header=None)
start_time = np.array(start_time.values.astype(float).flatten())

MISSING_DURATION = start_time[0]
start_time = start_time[1:].astype(int)

MISSING_LENGTH = int(MISSING_DURATION*SAMPLING_FREQUENCY)
MASK_SIZE = int(NOISE_DURATION*SAMPLING_FREQUENCY)

data = pd.read_csv(DATA_FILE, header=None)
data = data.values.astype(float)

X = np.repeat(data, 64, 0)
X = np.expand_dims(X, -1)
X = np.swapaxes(X, 0, 1)

model = imputationLSTM(INPUT_SIZE, HIDDEN_SIZE).to(DEVICE)
# summary(model, (3, 112, 112))

In [ ]:
def convertStartTimeToMask(start_time, missing_length, mask_size=600*4096):
    mask = np.zeros((start_time.shape[0], mask_size))

    for i in range(start_time.shape[0]):
        mask[i, start_time[i]:start_time[i]+missing_length] = 1

    return mask

def train(X, start_time, model, train_loss_arr, optimizer, batch_size=64):
    size = NO_OF_TRAINING_SAMPLES
    num_batches = math.ceil(size/batch_size)

    X = torch.tensor(X)
    start_time = torch.tensor(X)
    
    model.train()
    train_loss = 0

    for i in range(num_batches):
        if i != num_batches-1:
            time = start_time[i*batch_size : (i+1)*batch_size].to(DEVICE)

        else:
            time = start_time[i*batch_size :].to(DEVICE)

        X.requires_grad = True
        mask = convertStartTimeToMask(time, MISSING_LENGTH, MASK_SIZE)

        out, loss = model(X, mask)
        train_loss += loss.item()
        
        # For visualizing the model
        # make_dot((out, loss), params=dict(list(model.named_parameters()))).render("imputationLSTM", format="png")        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= num_batches
    print(f"\nTraining - Avg loss: {train_loss:>8f} \n")

    train_loss_arr.append(train_loss)

def test(X, start_time, model, test_loss_arr, batch_size=64):
    size = NO_OF_TRAINING_SAMPLES
    num_batches = math.ceil(size/batch_size)

    X = torch.tensor(X)
    start_time = torch.tensor(X)
    
    model.train()
    test_loss = 0

    with torch.no_grad():
        for i in range(num_batches):
            if i != num_batches-1:
                time = start_time[i*batch_size : (i+1)*batch_size].to(DEVICE)

            else:
                time = start_time[i*batch_size :].to(DEVICE)

            X.requires_grad = True
            mask = convertStartTimeToMask(time, MISSING_LENGTH, MASK_SIZE)

            out, loss = model(X, mask)
            test_loss += loss.item()

    test_loss /= num_batches
    print(f"Testing - Avg loss: {test_loss:>8f} \n")

    test_loss_arr.append(test_loss)    

In [ ]:
if LOAD_WEIGHTS:
    model.load_state_dict(torch.load(WEIGHTS_PATH))

optimizer = torch.optim.Adam(model.parameters(), lr=LR) # , weight_decay=cfg.weight_decay

train_loss_arr = []
test_loss_arr = []

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(X, start_time, model, train_loss_arr, optimizer)
    # test(test_loader, model, loss_fn_arr, test_loss_arr)
        
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S_")
torch.save(model.state_dict(), BASE_PATH + dt_string + f"weights.pth")

x = [i+1 for i in range(EPOCHS)]
plt.plot(x, train_loss_arr, 'g', label='train')
# plt.plot(x, test_loss_arr, 'r', label='test')
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.savefig(BASE_PATH + dt_string + "loss.png")